<a href="https://colab.research.google.com/github/gustavofg1pontes/web-scrapper/blob/main/webScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrapper

Nesse `notebook`, há duas versões:
uma é feita com base na quantidade de caracteres e retorna os parágrafos (tag `<p>` das páginas, com no mínimo 20 caracteres);
outra retorna todo o conteúdo.
ambos meios filtram em até 5 links, contando com o original.


## 1:

A decisão de ir até aprox. 3000 caracteres foi por conta da limitação dos prompts do ChatGPT, forma que usei para criar chunks temporários para os clientes - ou seja, que serão refinados posteriormente. O mínimo de caracteres nos parágrafos é para retirar o conteúdo "interativo" que vem da página, como `Home`, `Sobre`, etc...

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL do site que você deseja acessar
url = ''
links = []
consumed_links = [url]
max_carac = 3000
min_carac_p = 20

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
}

concatenated_text = ""

def fetch_and_parse(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException as e:
        print(f'Erro ao acessar {url}: {e}')
        return None

def scrape(url):
    global concatenated_text
    soup = fetch_and_parse(url)
    if not soup:
        return

    article_titles = soup.find_all(['p'])
    for title in article_titles:
        text = title.get_text(strip=True)
        if text != '' and len(text) > min_carac_p:
            concatenated_text += text + "\n"
        if len(concatenated_text) >= max_carac:
            return

    anchors = soup.find_all('a')
    for a in anchors:
        link = a.get('href')
        if len(links) >= 5:
            continue
        if link and link not in consumed_links and link not in links and url in link:
            links.append(link)

    print(f"##### {url}")

scrape(url)

if len(concatenated_text) < max_carac:
  for link in links:
      if link not in consumed_links:
          scrape(link)
          consumed_links.append(link)

if concatenated_text:
    print(concatenated_text)


Alguns resultados usando o seguinte prompt no ChatGPT 3.5:

Prompt:
```
Decompor o "Conteúdo" em proposições claras e simples, garantindo que sejam interpretáveis fora de contexto. Lembre-se de que "Conteúdo" será o conteúdo de vários sites, mas você precisa ocultar essa informação. Evite repetir frases ou proposições com o mesmo significado: Fale sobre o produto que o site está vendendo e siga estes passos:

1. Divida a frase composta em frases simples. Mantenha a redação original do texto sempre que possível.
2. Para qualquer entidade nomeada que seja acompanhada por informações descritivas adicionais, separe essa informação em uma proposição distinta.
3. Descontextualize a proposição adicionando modificadores necessários aos substantivos ou a frases inteiras e substituindo pronomes (por exemplo, "isso", "ele", "ela", "eles", "isto", "aquilo") pelo nome completo das entidades às quais se referem.
4. Apresente os resultados como uma lista de strings, formatada em JSON.

Entrada:
```

Resultados:
`https://www.phileodontologia.com.br/`
```
[
    "O e-mail de contato é phile@phileodontologia.com.br.",
    "O endereço da clínica é Av. Paulista, 1471 - Cj 1514, São Paulo - SP, CEP: 01311-200.",
    "O horário de atendimento é de segunda a sexta-feira, das 08h às 18h.",
    "Phile Odontologia visa proporcionar um tratamento de excelência.",
    "Phile Odontologia utiliza tecnologia de ponta.",
    "Phile Odontologia oferece atendimento humanizado.",
    "A equipe da Phile Odontologia é especializada em diversas áreas da Odontologia.",
    "A Phile Odontologia oferece um tratamento personalizado e completo em um único lugar.",
    "A clínica possui ortodontista especialista em Invisalign®.",
    "A Phile Odontologia realiza cirurgia bucomaxilofacial, ortognática, implantes, tratamento da ATM e fraturas da face.",
    "A clínica oferece serviços em endodontia, estética e dentística.",
    "A gerente administrativa é responsável pelos serviços administrativos da clínica.",
    "A clínica conta com especialistas em periodontia, dentística e estética.",
    "Phile Odontologia foi inaugurada em 2006 no bairro da Bela Vista, São Paulo.",
    "A clínica foi fundada por quatro dentistas: Dra. Flávia Samuel, Dra. Juliane Chaves, Dra. Liandra Chaves e Dra. Renata Arcaldi.",
    "Dra. Flávia Samuel, Dra. Juliane Chaves, Dra. Liandra Chaves e Dra. Renata Arcaldi se formaram na mesma turma da Faculdade de Odontologia da USP.",
    "A origem do nome 'Phile' vem da palavra grega 'Philia', que significa amor de amizade.",
    "Em 2011, Dr. Daniel Uyeda se juntou à equipe da Phile Odontologia.",
    "Dr. Daniel Uyeda é formado pela USP e trouxe conhecimento e experiência à equipe.",
    "Todos os profissionais da clínica são formados pela Faculdade de Odontologia da USP e realizaram cursos em centros de ensino renomados do país.",
    "A clínica possui especialistas em periodontia, ortodontia, endodontia, odontopediatria, dentística, próteses, implantes, cirurgia bucomaxilofacial e atendimento a pacientes com necessidades especiais.",
    "O atendimento multi-disciplinar na clínica permite uma visão completa dos pacientes.",
    "O atendimento na Phile Odontologia garante o equilíbrio entre estética, funcionalidade da mastigação e fonação.",
    "A clínica proporciona qualidade de vida aos pacientes.",
    "Phile Odontologia realiza troca de experiências e discussão de casos para oferecer opções de tratamento mais eficazes.",
    "A Phile Odontologia está localizada a 30 metros da Estação do Metrô Trianon-Masp.",
    "A clínica oferece facetas e lentes de contato.",
    "Phile Odontologia é especialista em Invisalign® e alinhadores transparentes.",
    "A clínica realiza tratamentos de ortodontia com aparelhos dentários.",
    "Phile Odontologia oferece endodontia, tratamento de canal.",
    "A clínica realiza periodontia, tratamento de gengiva.",
    "A Phile Odontologia trata disfunção têmporo-mandibular.",
    "A clínica oferece cirurgia e traumatologia bucomaxilofacial.",
    "Phile Odontologia realiza odontopediatria, tratamento para crianças.",
    "A clínica oferece atendimento a pacientes com necessidades especiais.",
    "Phile Odontologia utiliza laser na odontologia.",
    "A clínica oferece óxido nitroso para sedação consciente.",
    "Phile Odontologia realiza aplicação de toxina botulínica.",
    "A clínica utiliza scanner 3D intraoral iTero."
]
```
`https://www.drmarcelopimenta.com.br/`
```
[
    "O propósito é acolher cada paciente com sua história e expectativas.",
    "São oferecidas soluções que respeitam as características únicas de cada paciente.",
    "Os avanços da tecnologia digital são aliados a uma atualização científica constante.",
    "A equipe demonstra um coração sensível ao paciente.",
    "O escaneamento intraoral é um método de reprodução em 3D que filma e digitaliza toda a boca do paciente.",
    "O escaneamento intraoral garante mais acurácia e dispensa moldes em gesso.",
    "A técnica Digital Smile Design (DSD) permite prever os resultados finais do tratamento odontológico.",
    "O paciente é envolvido no planejamento do tratamento odontológico e se torna co-projetista de seu sorriso.",
    "A cirurgia de implante guiada por computador usa alta tecnologia para a instalação de implantes dentários.",
    "A cirurgia guiada por computador alcança maior eficácia, precisão, agilidade e conforto para o paciente.",
    "Dr. Marcelo Pimenta é graduado em Odontologia pela USP.",
    "Dr. Marcelo Pimenta é Especialista em Implantodontia pela Associação Brasileira de Odontologia.",
    "Dr. Marcelo Pimenta é Pós-graduado em Odontologia Estética pela Associação Paulista de Cirurgiões-Dentistas.",
    "Dr. Marcelo Pimenta é membro do CEPI-USP (Centro de Excelência em Prótese e Implante).",
    "Dr. Marcelo Pimenta é membro do ITI (International Team for Implantology).",
    "Dra. Taciana Anfe é graduada em Odontologia pela PUC (Pontifícia Universidade Católica) de Campinas.",
    "Dra. Taciana Anfe possui Mestrado e Doutorado em Dentística pela USP.",
    "Dra. Taciana Anfe é Pós-graduada em Harmonização Orofacial pela Associação Paulista de Cirurgiões-Dentistas.",
    "Dra. Laís Pizzato é graduada em Odontologia pela Universidade Federal do Rio Grande do Sul.",
    "Dra. Laís Pizzato possui Especialização e Mestrado em Endodontia pela USP.",
    "Dra. Laís Pizzato possui atualização em microscopia endodôntica.",
    "Dra. Laís Pizzato realizou intercâmbio acadêmico na Universidade de Michigan, nos Estados Unidos.",
    "Dra. Laís Pizzato publica trabalhos científicos em revistas internacionais de endodontia.",
    "Dra. Danielle Sá é graduada em Odontologia pela UNESP.",
    "Dra. Danielle Sá é formada no Curso Integrado de Reabilitação Oral pela Fundecto/USP.",
    "Dra. Danielle Sá possui Especialização em Ortodontia pela Sociedade Paulista de Ortodontia (SPO).",
    "Dra. Danielle Sá tem foco em pacientes adultos em processo de reabilitação oral.",
    "A clínica segue um protocolo rígido de biossegurança conforme as normas da Anvisa.",
    "A clínica realiza assepsia diária e esterilização de instrumentais.",
    "São utilizados exclusivamente os implantes Straumann e Neodent.",
    "Straumann é uma marca suíça líder global em implantes dentários.",
    "Neodent é a segunda maior empresa de implantes dentários do mundo.",
    "Straumann e Neodent são reconhecidas internacionalmente por qualidade e excelência.",
    "A naturalidade é um pilar dos tratamentos oferecidos.",
    "A naturalidade é garantida considerando as características únicas de cada indivíduo no planejamento e execução das técnicas.",
    "A clínica alia tecnologia, experiência, atualização científica constante e sensibilidade dos especialistas."
]
```



## 2:

O limite de 5 links é para não entrar num site que possui tantas âncoras que a execução fica infinita.

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL do site que você deseja acessar
url = ''
links = []
consumed_links = [url]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
}

def fetch_and_parse(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException as e:
        print(f'Erro ao acessar {url}: {e}')
        return None

def scrape(url):
    soup = fetch_and_parse(url)
    if not soup:
        return

    page_text = soup.get_text(separator='\n', strip=True)

    anchors = soup.find_all('a')
    for a in anchors:
        link = a.get('href')
        if len(links) >= 5:
          continue
        if link and link not in consumed_links and link not in links and url in link:
          links.append(link)
    print(f"##### {url}")
    print(page_text)


scrape(url)

for link in links:
  if link not in consumed_links:
    scrape(link)
  consumed_links.append(link)
